# Notebook for the development of module "weather -> photovoltaic output"
### If you develop some new function useful also in other situations, please put it in a dedicated .py file, then import it here

In [1]:
!pip install openmeteo_requests
!pip install requests-cache retry-requests
!pip install pandas

In [2]:
from data_classes import *
from weather_providers import *
#import numpy as np        #pip install numpy
#import pvlib              #pip install pvlib            #library to automatically convert weather params into pv output

#### Set system parameters here

In [6]:
my_location = Location(latitude=40.71, longitude=-74.01) # Example: New York
my_panel_orientation = PanelOrientation(surface_tilt=30, surface_azimuth=180) # South-facing
my_panel_specs = PanelSpecs(rated_power=400, temp_coefficient_power=-0.35, noct=44.0) # Temp coeff as %/C
my_system_params = SystemParameters(system_losses=0.90, inverter_efficiency=0.97) # 10% system loss
print(f"System Params: {my_system_params}")
print(f"Location: {my_location}")
print(f"Orientation: {my_panel_orientation}")
print(f"Panel Specs: {my_panel_specs}")

System Params: System Losses: 10.0% (Factor: 0.900), Inverter Eff: 97.0% (Factor: 0.970)
Location: Lat: 40.7100, Lon: -74.0100, Alt: not defined
Orientation: Tilt: 30.0°, Azimuth: 180.0°
Panel Specs: Pmax: 400.0Wp, TempCoeff: -35.000%/°C, NOCT: 44.0°C


#### Weather request start and end time

In [9]:
start_dt = datetime(2025, 3, 1, 0, 0, 0, tzinfo=timezone.utc) # March 1st, 00:00 UTC
end_dt = datetime(2025, 4, 14, 23, 59, 59, tzinfo=timezone.utc) # April 14th, end of day UTC

#### Call to open-weather

In [10]:
try:
    weather_provider = get_weather_provider("open-meteo") # Factory returns the correct provider

    print(f"\n--- Requesting Hourly DataFrame ---")
    print(f"Fetching hourly weather data for {my_location} from {start_dt} to {end_dt}...")

    weather_df: pd.DataFrame | None = weather_provider.get_weather_data(my_location, start_dt, end_dt)

    if weather_df is None:
        print("Could not retrieve weather data DataFrame.")
    elif weather_df.empty:
         print("Retrieved DataFrame is empty (no data for the specified range/parameters).")
    else:
        print(f"\nSuccessfully retrieved DataFrame with {len(weather_df)} hourly rows.")
        print("\nDataFrame Info:")
        weather_df.info()
        print("\nDataFrame Head:")
        print(weather_df.head())
        print("\nDataFrame Tail:")
        print(weather_df.tail())

        # --- Subsequent Processing ---
        # Now you can directly use the DataFrame for analysis or PV calculations
        # Example: Accessing a column
        # temperatures = weather_df['temperature_2m']
        # print("\nSample Temperatures:\n", temperatures.head())

        # Example: Passing data to a PV calculation function (needs adaptation)
        # pv_output = calculate_pv_power_from_df(weather_df, my_location, panel_orientation, ...)


except ValueError as e:
    print(f"Configuration Error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    import traceback
    traceback.print_exc()

OpenMeteo client initialized.

--- Requesting Hourly DataFrame ---
Fetching hourly weather data for Lat: 40.7100, Lon: -74.0100, Alt: not defined from 2025-03-01 00:00:00+00:00 to 2025-04-14 23:59:59+00:00...
start time: 2025-03-01, end time: 2025-04-14
Requesting Open-Meteo (Hourly) via library: {'latitude': 40.71, 'longitude': -74.01, 'timezone': 'UTC', 'hourly': ['temperature_2m', 'precipitation_probability', 'wind_speed_10m', 'shortwave_radiation_instant', 'diffuse_radiation_instant', 'direct_normal_irradiance_instant', 'cloud_cover', 'precipitation'], 'start_date': '2025-03-01', 'end_date': '2025-04-14'}
parsing:  temperature_2m
parsing:  precipitation_probability
parsing:  wind_speed_10m
parsing:  shortwave_radiation_instant
parsing:  diffuse_radiation_instant
parsing:  direct_normal_irradiance_instant
parsing:  cloud_cover
parsing:  precipitation

Successfully retrieved DataFrame with 1080 hourly rows.

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1080 en

In [ ]:
HOURLY_VARS = [
        "temperature_2m",
        "precipitation_probability",
        "wind_speed_10m",
        "shortwave_radiation_instant",    # Instant GHI
        "diffuse_radiation_instant",      # Instant DHI
        "direct_normal_irradiance_instant", # Instant DNI
        "cloud_cover",
        "precipitation"
    ]

latitude, longitude,
date (hour)

wind speed -> cloud cover -> precipitation probability -> precipitation -> GHI
                          
                          

In [ ]:
from BN import *

# --- Example Usage ---

my_lat = 40.71
my_lon = -74.01
my_tilt = 30
my_azimuth = 180

# Example Input Data point (Clear, Sunny Day around Noon)
timestamp = datetime(2024, 7, 15, 12, 0, 0, tzinfo=timezone.utc)
input_data = {
    "temp_air": 28.0,
    "wind_speed": 2.5,
    "ghi": 900.0, # Used for external consistency calc
    "dhi": 100.0, # Used for external consistency calc
    "dni": 850.0, # Used for external consistency calc
    "cloud_cover": 10.0, # BN Evidence
    "precip_amount": 0.0  # BN Evidence
}

accuracy_estimate = estimate_accuracy_continuous_weather(
    lat=my_lat,
    lon=my_lon,
    surface_tilt=my_tilt,
    surface_azimuth=my_azimuth,
    dt_utc=timestamp,
    **input_data # temp_air, wind_speed, ghi, dhi, dni, cloud_cover, precip_amount
)

if accuracy_estimate:
     print(f"\nEstimated Accuracy (Clear Day): Mean = {accuracy_estimate['mean']:.3f}, Std Dev = {accuracy_estimate['std_dev']:.3f}")
else:
     print("\nFailed to estimate accuracy (Clear Day).")


# Example Input Data point (Cloudy, Rainy Day)
timestamp_rainy = datetime(2024, 7, 15, 14, 0, 0, tzinfo=timezone.utc)
input_data_rainy = {
    "temp_air": 20.0,
    "wind_speed": 4.0,
    "ghi": 200.0,
    "dhi": 180.0,
    "dni": 20.0,
    "cloud_cover": 90.0,
    "precip_amount": 2.5
}

accuracy_estimate_rainy = estimate_accuracy_continuous_weather(
    lat=my_lat,
    lon=my_lon,
    surface_tilt=my_tilt,
    surface_azimuth=my_azimuth,
    dt_utc=timestamp_rainy,
    **input_data_rainy
)

if accuracy_estimate_rainy:
     print(f"\nEstimated Accuracy (Rainy Day): Mean = {accuracy_estimate_rainy['mean']:.3f}, Std Dev = {accuracy_estimate_rainy['std_dev']:.3f}")
else:
     print("\nFailed to estimate accuracy (Rainy Day).")